In [2]:
import pandas as pd
import json
import os

df = pd.read_csv("../data/results/all_dependencies.csv")

df["final_rating"]

results_file = "../data/results/without/all_dependencies_without_gpt-3.5-turbo-0125.json"


def transform_result_file(results_file: str):

    parts = results_file.split("/")    
    config = parts[3]
    llm_name = parts[-1].split("_")[-1].split(".json")[0]


    with open(results_file, "r", encoding="utf-8") as src:
        data = json.load(src)

    for entry in data:
        gt = df.iloc[entry["index"]]

        try:
            response_dict = json.loads(entry["response"], strict=False)
            entry["response_dict"] = response_dict
            entry["reasoning"] = response_dict["rationale"]
            entry["uncertainty"] = response_dict["uncertainty"]
    

            if gt["final_rating"] == response_dict["isDependency"]:
                if gt["final_rating"]:
                    entry["correct"] = "TP"
                else:
                    entry["correct"] = "TN"
            else:
                if gt["final_rating"]:
                    entry["correct"] = "FN"
                else:
                    entry["correct"] = "FP"

        except:
            entry["response_dict"] = entry["response"]
            entry["reasoning"] = None
            entry["uncertainty"] = None
            entry["correct"] = None

        entry["final_category"] = gt["final_category"]
        entry["sub_category"] = gt["sub_category"]
        entry["llm"] = llm_name
        entry["config"] = config


    with open(results_file, "w", encoding="utf-8") as dest:
        json.dump(data, dest, indent=2)

In [3]:
results_path = "../data/results/"
folder_paths = [results_path + x + "/" for x in os.listdir(results_path) if ".csv" not in x]

files = []

for path in folder_paths:
    for result_file in [path + x for x in os.listdir(path) if ".json" in x]:
        files.append(result_file)

In [4]:
for file in files:
    transform_result_file(results_file=file)

In [5]:
df = pd.DataFrame()

for file in files:
    with open(file, "r", encoding="utf-8") as src:
        data = json.load(src)
    
    df = pd.concat([df, pd.DataFrame(data)], axis=0, ignore_index=True)

columns = ["llm", "config", "uncertainty", "correct", "reasoning", "context_str", "project", "option_name", "option_type", "option_value","option_technology", 
           "option_file", "dependent_option_name", "dependent_option_type", "dependent_option_value",
           "dependent_option_technology", "dependent_option_file", "final_category", "sub_category"]

json_df = pd.json_normalize(df["dependency"])

df = pd.concat([df.drop("dependency", axis=1), json_df], axis=1)

df = df[columns]
df_2 = df
df = df[df["correct"].isin(["FP", "FN"])]

df = df[df["config"].isin(["without", "config2"])]

df = df[df["llm"].isin(['gpt-4o-2024-05-13', 'llama3:70b', 'llama3:8b', 'gpt-3.5-turbo-0125'])]

df.to_csv("../data/analysis/failures.csv")

In [9]:
df

0         True
6         True
12        True
19        True
20        True
         ...  
15481    False
15484    False
15485    False
15486    False
15494    False
Name: config, Length: 1186, dtype: bool

In [ ]:
df["uncertainty"].describe()

In [ ]:
from scipy.stats import ttest_ind

df_2 = df_2[df_2["correct"].isin(["TP", "TN"])]
df_2 = df_2[df_2["config"].isin(["without", "config2"])]
df_2 = df_2[df_2["llm"].isin(['gpt-4o-2024-05-13', 'llama3:70b', 'llama3:8b', 'gpt-3.5-turbo-0125'])]

df_2["uncertainty"].describe()

#ttest_ind(df["uncertainty"], df_2["uncertainty"])

count    2696.000000
mean        7.976261
std         1.255560
min         0.000000
25%         8.000000
50%         8.000000
75%         9.000000
max        10.000000
Name: uncertainty, dtype: float64

In [13]:
import pandas as pd


#df = pd.read_csv("../data/analysis/failures_annotated.csv")

df = pd.read_csv("../data/results/all_dependencies.csv")


dup_columns = ["option_value", "option_name", "option_file", "dependent_option_name", "dependent_option_value", "dependent_option_file"]

duplicates = df[df.duplicated(subset=dup_columns)]

for index, row in duplicates.iterrows():
    print(row["link_str"])

print(len(df))

df = df.drop_duplicates(subset=dup_columns, keep="first")


print(len(df))

duplicates.index

litemall::::litemall-admin-api/pom.xml::::project::::dependencies::::dependency_litemall-db::::groupId::::org.linlinjava <-> litemall::::litemall-all-war/pom.xml::::project::::dependencies::::dependency_litemall-wx-api::::groupId::::org.linlinjava
litemall::::litemall-wx-api/pom.xml::::project::::parent_litemall::::version::::version:0.1.0 <-> litemall::::pom.xml::::project::::version::::version:0.1.0
mall-swarm::::mall-mbg/pom.xml::::project::::artifactId::::mall-mbg <-> mall-swarm::::mall-portal/pom.xml::::project::::dependencies::::dependency_mall-mbg::::artifactId::::mall-mbg
mall-swarm::::mall-portal/pom.xml::::project::::artifactId::::mall-portal <-> mall-swarm::::pom.xml::::project::::modules::::module::::mall-portal
mall::::document/sh/Dockerfile::::ADD::::src::::mall-admin-1.0-SNAPSHOT.jar <-> mall::::document/sh/Dockerfile::::ENTRYPOINT::::param2::::mall-admin-1.0-SNAPSHOT.jar
music-website::::music-server/src/main/resources/application-dev.properties::::spring.datasource.use

Index([125, 133, 181, 192, 238, 264, 268, 282, 285, 486], dtype='int64')

In [14]:
df = pd.read_csv("../data/results/all_dependencies_updated.csv")


dup_columns = ["option_value", "option_name", "option_file", "dependent_option_name", "dependent_option_value", "dependent_option_file"]

duplicates = df[df.duplicated(subset=dup_columns)]

for index, row in duplicates.iterrows():
    print(row["link_str"])

print(len(df))

df = df.drop_duplicates(subset=dup_columns, keep="first")


print(len(df))

duplicates.index

500
500


Index([], dtype='int64')